# Unlocking AI Agents for Network Engineering

This (Jupyter Notebook) is an interactive coding environment that lets you write and run Python code in a web-based interface. Simply click on the play button or [SHIFT]+[ENTER] to execute the cell.

## 1. Connectivity

### Catalyst Router (IOS XE)

In this lab we use the Catalyst 8000V IOS XE sandbox. Alternatively, you can also use the always-on sandbox.

1. Connect with your VPN client to the sandbox environment.
2. Open a terminal on the device and try to run a show-command to see if it works.
3. During the lab the credentials are stored in the **hosts.json** file to simulate external credentials retrieval.


**Hosts.json file content:**

In [1]:
{
    "10.10.20.48" : {
            "type" : "cisco catalyst ios xe",
            "username" : "developer",
            "password" : "C1sco12345"
        },
    "devnetsandboxiosxe.cisco.com" : {
            "type" : "cisco catalyst ios xe",
            "username" : "admin",
            "password" : "C1sco12345"
        }
}

{'10.10.20.48': {'type': 'cisco catalyst ios xe',
  'username': 'developer',
  'password': 'C1sco12345'},
 'devnetsandboxiosxe.cisco.com': {'type': 'cisco catalyst ios xe',
  'username': 'admin',
  'password': 'C1sco12345'}}

### Local Large Language Model (Ollama)

Let's see how the local large language model is working and the duration time of the prompt execution.

In [2]:
from ollama import chat
from ollama import ChatResponse

# LLM input = system prompt + user prompt
response: ChatResponse = chat(model = 'llama3.1', messages = [
    {
        'role': 'system',
        'content': 'You are a helpful networking assistant.',
    },
    {
        'role': 'user',
        'content': 'How can I show all routes on a Cisco switch?',
    }
])

# output
print(response['message']['content'])

To show all routes on a Cisco switch, you can use the `show ip route` command.

Here's how to do it:

1. Log in to your Cisco switch using your credentials.
2. Access the privileged EXEC mode by typing `enable` and entering your enable password or secret key.
3. Type `show ip route` in the command-line interface (CLI) to display all routes on the switch.

Alternatively, you can use other commands like:

* `show ip route vrf <vrf-name>`: To show routes for a specific VRF (Virtual Private Network).
* `show ip route summary`: To summarize and display the routing table in a concise format.
* `show ip route detail`: To display detailed information about each route.

Note that you need to be in privileged EXEC mode (`#`) to execute these commands. If you're not sure which mode you're in, you can check by typing `sh prompt` or simply looking for the `#` symbol at the end of the command line.


## 2. Create Your First AI Agent with `smolagents`

Let's start writing our first AI agent in this framework and learn more about its components.

In [3]:
from smolagents.agents import CodeAgent
from smolagents import LiteLLMModel

model = LiteLLMModel(model_id="ollama/qwen2.5",
                     num_ctx=8192
                )

device_agent = CodeAgent(
                tools=[],
                model=model
)

device_agent.run("What does sh ip route do?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What does sh ip route do?                                                                                       │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5 ─────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The 'sh ip route' command in Cisco devices displays the current routing table. It shows which      
  network routes are currently active and available for packets to be forwarded to their destinations. This        
  includes directly connected networks (those with a Direct or Null route) as well as learned routes through       
  dynamic routing protocols like OSPF, EIGRP, etc., or static routes.")                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The 'sh ip route' command in Cisco devices displays the current routing table. It shows which 
network routes are currently active and available for packets to be forwarded to their destinations. This includes 
directly connected networks (those with a Direct or Null route) as well as learned routes through dynamic routing 
protocols like OSPF, EIGRP, etc., or static routes.

[Step 0: Duration 21.84 seconds| Input tokens: 2,276 | Output tokens: 144]

"The 'sh ip route' command in Cisco devices displays the current routing table. It shows which network routes are currently active and available for packets to be forwarded to their destinations. This includes directly connected networks (those with a Direct or Null route) as well as learned routes through dynamic routing protocols like OSPF, EIGRP, etc., or static routes."

## 3. Important Building Blocks for AI Agents

In order to understand AI agents, it is important to know these building blocks of AI agentic frameworks:

* Understanding the Agentic Behavior using OpenTelemetry
* Function/Tool Calling
* Structured Output

### 3.1 Using OpenTelemetry for Understanding & Debugging

Start the OpenTelemetry server, access it at [http://0.0.0.0:6006/projects/](http://0.0.0.0:6006/projects/) and run this code below. The next time you are executing a smolagent, you will see data.

In [4]:
# ================== TELEMETRY ==================
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
endpoint = "http://0.0.0.0:6006/v1/traces"
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

### 3.2. Function/Tool Calling with LLMs

Let's extend our code and define these custom functions. Execute the following code which does not provide any output.

In [5]:
from smolagents import tool

@tool
def get_username_password_for_device(ip_address:str) -> str:
    """
    Returns the username and password separated by a comma for the given ip address or hostname.

    Args:
        ip_address: The IP address or hostname of the device

    Returns:
        str: A string with the username and password separated by a comma.
    """
    import json
    
    with open("hosts.json", 'r') as file:
        devices = json.load(file)
        return f"{devices[ip_address]['username']},{devices[ip_address]['password']}"

@tool
def show_ip_route(host:str,username:str,password:str,device_type:str = "cisco_ios",) -> str:
    """
    Return the routing table of the device. Executes the command 'show ip route'.

    Args:
        host: The IP address or hostname of the device
        username: The username for the device
        password: The password for the device
        device_type: The device type for the device

    Returns:
        str: The routing table of the device.
    """
    from netmiko import ConnectHandler    
                                                                                                          
    device = {                                                                                                                                                                 
        'ip': host,                                                                                                                                                   
        'username': username,                                                                                                                                                  
        'password': password,                                                                                                                                                  
        'device_type': device_type                                                                                        
    }                                                                                                                                                                          
                                                                                                                                                                                
    # Establish connection to the device                                                                                                                                       
    connection = ConnectHandler(**device)                                                                                                                                                                                                                                                                                     
                                                                                                                                                                                
    # Execute command to retrieve running configuration                                                                                                                        
    running_config_output = connection.send_command('show ip route')                                                                                                     
                                                                                                                                                                                
    return running_config_output

Now, let's add these functions to the CodeAgent and authorize to import these Python libraries in the sandboxed Python environment of the agent. Let's run this example with the following prompt.

In [6]:
from smolagents.agents import CodeAgent
from smolagents import LiteLLMModel

model = LiteLLMModel(model_id="ollama/qwen2.5",
                     num_ctx=8192)

device_agent = CodeAgent(tools=[get_username_password_for_device,
                         show_ip_route],
                  model=model,
                  additional_authorized_imports=['ncclient', 'netmiko','requests','paramiko','io'],
                 )
# give the sandboxed Python interpreter access to read/write files outside (use with caution!)
device_agent.python_executor.static_tools["open"] = open

# Routing Table Summary as Markdown
device_agent.run("""
                 Summarize the routing table on the Cisco device 10.10.20.48.
                 Save it in the Markdown file 'routing_table_summary.md'.
                 You will at first need the username and password for the device.
                 """)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Summarize the routing table on the Cisco device 10.10.20.48.                                                    │
│                  Save it in the Markdown file 'routing_table_summary.md'.                                       │
│                  You will at first need the username and password for the device.                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5 ─────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  device_ip = "10.10.20.48"                                                                                        
  credentials = get_username_password_for_device(ip_address=device_ip)                                             
  print(f"Username: {credentials.split(',')[0]}, Password: {credentials.split(',')[1]}")                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Username: developer, Password: C1sco12345

Out: None

[Step 0: Duration 9.13 seconds| Input tokens: 2,563 | Output tokens: 115]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Define the routing table content as a string                                                                   
  routing_table_content = """                                                                                      
  Gateway of last resort is 10.10.20.254 to network 0.0.0.0                                                        
                                                                                                                   
  S*    0.0.0.0/0 [1/0] via 10.10.20.254, GigabitEthernet1                                                         
        10.0.0.0/8 is variably subnetted, 6 subnets, 2 masks                                                       
  C        10.0.0.0/24 is directly connected, Loopback0                                                            
  L        10.0.0.1/32 is directly connected, Loopback0                                                            
  C        10.10.20.0/24 is directly connected, GigabitEthernet1                                                   
  L        10.10.20.48/32 is directly connected, GigabitEthernet1                                                  
  C        10.255.255.0/24 is directly connected, Loopback109                                                      
  L        10.255.255.9/32 is directly connected, Loopback109                                                      
        192.168.1.0/24 is variably subnetted, 2 subnets, 2 masks                                                   
  C        192.168.1.0/24 is directly connected, VirtualPortGroup0                                                 
  L        192.168.1.1/32 is directly connected, VirtualPortGroup0                                                 
  """                                                                                                              
                                                                                                                   
  # Save the routing table content to a Markdown file                                                              
  with open('routing_table_summary.md', 'w') as f:                                                                 
      f.write("# Routing Table Summary\n")                                                                         
      f.write(routing_table_content)                                                                               
                                                                                                                   
  final_answer("Summary of the routing table has been saved in routing_table_summary.md.")                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Summary of the routing table has been saved in routing_table_summary.md.

[Step 2: Duration 56.10 seconds| Input tokens: 9,155 | Output tokens: 1,132]

'Summary of the routing table has been saved in routing_table_summary.md.'

**Expected results**: After execution, the LLM intelligently calls `get_username_password_for_device` first and then `show_running_configuration` based on your prompt. You should see the summary of the routing table in the newly created file `routing_table_summary.md` which should be located in our project folder.

### 3.3 Structured Output with LLMs

Let's understand structured outputs with the following code. At first, we define just one function which returns a show command on the device. Execute this cell, there should be no output.

In [7]:
def run_ios_show_command_on_device_trusted(show_command:str,host:str) -> str:
    """
    Returns the output of the provided show command from the device with the given host, username, password, and device type.

    Args:
        show_command: The show command to run on the device
        host: The IP address or hostname of the device
        device_type: The device type for the device

    Returns:
        str: Output of the provided show command
    """
    print("Running show command on device...")

    from netmiko import ConnectHandler
    import json

    with open("hosts.json", 'r') as file:
        devices = json.load(file)
        username = devices[host]["username"]
        password = devices[host]["password"]
                                                          
    device = {                                                                                                                                                                 
        'ip': host,                                                                                                                                                   
        'username': username,                                                                                                                                                  
        'password': password,                                                                                                                                                  
        'device_type': "cisco_ios",                                                                                        
    }                                                                                                                                                                          
                                                                                                                                                                                
    # Establish connection to the device                                                                                                                                       
    connection = ConnectHandler(**device)

    if show_command.startswith("show") or show_command.startswith("sh"):
        # Execute command to retrieve running configuration                                                                                                                        
        running_config_output = connection.send_command(show_command)
        print(running_config_output)
        return running_config_output
    else:
        return "Error! You are only allowed to run show commands. Try again and use a show command."

Then, let's define the output schema at first, the parameters for our prompt:

In [8]:
from pydantic import BaseModel

# here we define the model for the response
class Cat8000(BaseModel):
    ios_version: str
    configuration_register: str

# define your parameters
ios_show_command = "show version"
device_host = "10.10.20.48" # or "devnetsandboxiosxe.cisco.com"

Then let's define our prompts: system + user prompt and tell the LLM how to structure the output and run the query:

In [11]:
from ollama import chat

response = chat(
    messages=[
        {
        'role': 'system',
        'content': 'You are a helpful networking assistant.',
    },
    {
        'role': 'user',
        'content': f'''What can you say about my Cisco switch?
                    Here is the output of the "show version" command:
                    {run_ios_show_command_on_device_trusted(ios_show_command,device_host)}''',
    }
    ],
    model='llama3.1',
    format=Cat8000.model_json_schema(),
)

cat8000_instance = Cat8000.model_validate_json(response.message.content)

Running show command on device...
Cisco IOS XE Software, Version 17.12.02
Cisco IOS Software [Dublin], Virtual XE Software (X86_64_LINUX_IOSD-UNIVERSALK9-M), Version 17.12.2, RELEASE SOFTWARE (fc2)
Technical Support: http://www.cisco.com/techsupport
Copyright (c) 1986-2023 by Cisco Systems, Inc.
Compiled Tue 14-Nov-23 05:59 by mcpre


Cisco IOS-XE software, Copyright (c) 2005-2023 by cisco Systems, Inc.
All rights reserved.  Certain components of Cisco IOS-XE software are
licensed under the GNU General Public License ("GPL") Version 2.0.  The
software code licensed under GPL Version 2.0 is free software that comes
with ABSOLUTELY NO WARRANTY.  You can redistribute and/or modify such
GPL code under the terms of GPL Version 2.0.  For more details, see the
documentation or "License Notice" file accompanying the IOS-XE software,
or the applicable URL provided on the flyer accompanying the IOS-XE
software.


ROM: IOS-XE ROMMON

cat8000v uptime is 4 days, 12 hours, 33 minutes
Uptime for this

In [10]:
print("LLM response output according to your schema:")
print(f"IOS Version: {cat8000_instance.ios_version}")
print(f"Config Register: {cat8000_instance.configuration_register}")

LLM response output according to your schema:
IOS Version: 17.12.02
Config Register: 0x2102


You will see that the LLM is extracting information from the `show version` command according to your define schema.

# Next steps

Check out the use-cases which are stored in the folder **complete examples**! Run them and go through the code to understand the concept and how you can apply them to your environment and use-cases.